# Redes Neurais Convolucionais

Vamos agora implementar a rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das redes que trouxeram todo esse interesse para a área de *deep learning*.





## Configuração do ambiente

In [ ]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchinfo import summary

import time, os, sys, numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

## Carregamento da base da dados

A função `load_data_cifar10` carrega e prepara o dataset CIFAR-10 para treinamento e teste.

CIFAR-10 é um conjunto de dados amplamente utilizado em tarefas de visão computacional e aprendizado de máquina, especialmente em problemas de classificação de imagens. Ele consiste em imagens coloridas de dimensão 32 x 32, separadas em 10 classes.

In [ ]:
def load_data_cifar10(batch_size, resize=None, root='/pgeoprj2/ciag2024/dados/'):
    root = os.path.expanduser(root)

    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    cifar10_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=transformer)
    cifar10_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=transformer)

    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(cifar10_train,
                                            batch_size, shuffle=True,
                                            num_workers=num_workers)

    test_iter = torch.utils.data.DataLoader(cifar10_test,
                                            batch_size, shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter

## Funções auxiliares

* `evaluate_accuracy` calcula a acurácia de um modelo em um dataset.

* `train_validate` implemneta o treinamento e validação de uma rede.

In [ ]:
def evaluate_accuracy(data_iter, net, loss):
    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()
    with torch.no_grad():

      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)

def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs):
    print('training on', device)

    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)

            trainer.zero_grad()
            l = loss(y_hat, y).sum()

            l.backward()
            trainer.step()

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]

        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)

        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))

## AlexNet

Agora já temos todo o conhecimento necessário para implementar nossa primeira arquitetura moderna.

Vamos implementar a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das arquiteturas mais famosas dessa nova onda de rede neurais.

<p align="center">
  <img width=700 src="https://www.researchgate.net/profile/Jaime_Gallego2/publication/318168077/figure/fig1/AS:578190894927872@1514862859810/AlexNet-CNN-architecture-layers.png">
</p>

<p align="center">
  <img width=700 src="https://miro.medium.com/max/700/1*vXBvV_Unz3JAxytc5iSeoQ.png">
</p>

### Atividade

Implementa sua rede neural baseada na AlexNet.

Lembre-se que, após cada camada de convolução e linear, há uma ativação não linear ReLU.

**Dica:** utilize blocos *sequential* para diminuir a complexidade da função de forward.

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        # Implemente aqui a arquitetura da rede

    def forward(self, x):
        # Implemente aqui a função de forward
        return out

In [ ]:
num_epochs, lr, batch_size, weight_decay = 20, 0.001, 100, 0.0001

net = # Instancie a rede (lembre-se de passar os parâmetros input_channels e classes)
net.to(device)
print(summary(net,(batch_size, 3, 227, 227)))

loss = # Defina a função de custo

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)